## Initialize environment

In [ ]:
from dotenv import load_dotenv

load_dotenv(dotenv_path=".env")

## Initialize Embeddings

We use AWS Bedrock Embedding

In [ ]:
import os
from langchain_community.embeddings import BedrockEmbeddings
from langchain_community.vectorstores.azuresearch import AzureSearch


class AWSBedrockEmbeddings:

    def __init__(self):
        self._embeddings = None
        self._validate_aws_env_variables()
        self._region_name = os.environ["AWS_REGION"]
        self._model_id = os.environ["AWS_LLM_EMBEDDINGS_ID"]
        self.initialize_embeddings()

    def initialize_embeddings(self):
        self._embeddings = BedrockEmbeddings(region_name=self._region_name, model_id=self._model_id)

    @property
    def region_name(self):
        return self._region_name

    @property
    def model_id(self):
        return self._model_id

    @property
    def embeddings(self):
        return self._embeddings

    def _validate_aws_env_variables(self):
        if "AWS_REGION" not in os.environ:
            raise ValueError("AWS_REGION environment variable not set")
        if "AWS_LLM_EMBEDDINGS_ID" not in os.environ:
            raise ValueError("AWS_LLM_EMBEDDINGS_ID environment variable not set")
        if "AWS_ACCESS_KEY_ID" not in os.environ:
            raise ValueError("AWS_ACCESS_KEY_ID environment variable not set")
        if "AWS_SECRET_ACCESS_KEY" not in os.environ:
            raise ValueError("AWS_SECRET_ACCESS_KEY environment variable not set")

embeddings = AWSBedrockEmbeddings().embeddings

## Initialize Vector Store

We use PGVector as our vector store to load the embeddings.

[Langchain Documentation Reference](https://python.langchain.com/docs/integrations/vectorstores/pgvector)

In [ ]:
from langchain_community.vectorstores.pgvector import PGVector

CONNECTION_STRING = PGVector.connection_string_from_db_params(
    driver=os.environ.get("PGVECTOR_DRIVER", "psycopg2"),
    host=os.environ.get("POSTGRES_HOST", "localhost"),
    port=int(os.environ.get("POSTGRES_PORT", "5432")),
    database=os.environ.get("POSTGRES_DB", "postgres"),
    user=os.environ.get("POSTGRES_USER", "postgres"),
    password=os.environ.get("POSTGRES_PASSWORD", "postgres"),
)

## Load data into the vector store

We load the [payment receipt](./Payment_Receipt.pdf) file into the vector store.

### Split the pdf data via RecursiveCharacterTextSplitter

In [ ]:
from langchain.document_loaders import  PDFPlumberLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

pdf_loader = PDFPlumberLoader("./Payment_Receipt.pdf")
pdf_data = pdf_loader.load()

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 500,
    chunk_overlap = 50,
    length_function = len,
)

docs = text_splitter.split_documents(pdf_data)

### Load the data into the PgVector vector store

In [ ]:
# The PGVector Module will try to create a table with the name of the collection.
# So, make sure that the collection name is unique and the user has the permission to create a table.

COLLECTION_NAME = "pgvector_exploration"

if docs:
    vector_store = PGVector(
        embedding_function=embeddings,
        collection_name=COLLECTION_NAME,
        connection_string=CONNECTION_STRING,
    )
    vector_store.add_documents(docs)
    print("Successfully ingested the data into the vector store.")

### Similarity Search with Cosine Distance (Default)

In [ ]:
query = "what is the mode of payment ?"

docs_with_score = vector_store.similarity_search_with_score(query, k=2)

for doc, score in docs_with_score:
    print("-" * 80)
    print("Cosine Distance (Score): ", score)
    print("Cosine Similarity (Score): ", 1 -score)  # Cosine Similarity = 1 - Cosine Distance
    print(doc.page_content)
    print("-" * 80)

### Maximal Marginal Relevance Search (MMR)

Maximal marginal relevance optimizes for similarity to query AND diversity among selected documents.

In [ ]:
docs_with_score = vector_store.max_marginal_relevance_search_with_score(query, k=2)

for doc, score in docs_with_score:
    print("-" * 80)
    print("Cosine Distance (Score): ", score)
    print("Cosine Similarity (Score): ", 1 -score)  # Cosine Similarity = 1 - Cosine Distance
    print(doc.page_content)
    print("-" * 80)

### Using vector store as retriever

In [ ]:
retriever = vector_store.as_retriever(search_type = "similarity", search_kwargs={"k": 2})

docs = retriever.invoke(query)

print(docs)

## LLM invocation with Pgvector as vector store

In [ ]:
from langchain import hub
from langchain_openai import AzureChatOpenAI
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

# Prepare prompt template from langsmith hub
prompt = hub.pull("rlm/rag-prompt")

# Initialize Chat Model
azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
deployment_name = os.getenv("AZURE_LLM_MODEL_DEPLOYMENT_NAME")
openai_api_key = os.getenv("AZURE_API_KEY")
openai_api_version= os.getenv("AZURE_API_VERSION")

llm = AzureChatOpenAI(azure_endpoint=azure_endpoint, 
                  deployment_name=deployment_name,
                  openai_api_key=openai_api_key,
                  openai_api_version=openai_api_version,
                  temperature=0)

In [ ]:
# Merge documents retrieved from vector store.
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [ ]:
query = "what is the payment amount ?"

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

chunks = []
for chunk in rag_chain.stream(query):
    chunks.append(chunk)
    print(chunk, end="", flush=True)